In [ ]:
import numpy as np
from keras import layers
from keras.layers import Dense, Conv2D,BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import keras
import numpy as np
import os

In [ ]:
batch_size=32
epochs=200
data_augmentation =True
num_classes=10

subtract_pixel_mean=True
n=3

version=1

if version==1:
  depth=n*6+2
elif version==2:
  depth=n*9+2

model_type='ResNet % dv % d' % (depth,version)

(x_train,y_train),(x_test,y_test)= cifar10.load_data()

input_shape=x_train.shape[1:]

x_train=x_train.astype('float32')/255
x_test=x_test.astype('float32')/255

if(subtract_pixel_mean):
  x_train_mean=np.mean(x_train,axis=0)
  x_train-=x_train_mean
  x_test-=x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0],'train samples')
print(x_test.shape[0],'test samples')
print('y_train shape:',y_train.shape)

y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

170498071/170498071 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [ ]:
def lr_schedule(epoch):
  lr=1e-3
  if epoch >180:
    lr*=0.5e-3
  elif epoch>160:
    lr*=1e-3
  elif epoch>120:
    lr*=1e-2
  elif epoch>80:
    lr*=1e-1
  print('Learning rate: ', lr)
  return lr

In [ ]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
  conv=Conv2D(num_filters,
              kernel_size=kernel_size,
              strides=strides,
              padding='same',
              kernel_initializer='he_normal',
              kernel_regularizer=l2(1e-4))
  x=inputs
  if conv_first:
    x=conv(x)
    if batch_normalization:
      x=BatchNormalization()(x)
    if activation is not None:
      x=Activation(activation)(x)
  else:
    if batch_normalization:
      x=BatchNormalization()(x)
    if activation is not None:
      x=Activation(activation)(x)
    x=conv(x)
  return x

In [ ]:
def resnet_v1(input_shape, depth, num_classes=10):
  if (depth-2)%6!=0:
    raise ValueError ('depth should be 6n+2(eg 20, 32, 44 in [a])')

  num_filters=16
  num_res_block=int ((depth-2)/6)

  inputs=Input(shape=input_shape)
  x=resnet_layer(inputs=inputs)

  for stack in range(3):
    for res_block in range(num_res_block):
      strides=1
      if stack > 0 and res_block ==0:
        strides = 2
      y=resnet_layer(inputs=x,
                     num_filters=num_filters,
                     strides=strides)
      y=resnet_layer(inputs=y,
                     num_filters=num_filters,
                     activation=None)
      if stack > 0 and res_block==0:
        x=resnet_layer(inputs=x,
                       num_filters=num_filters,
                       kernel_size=1,
                       strides=strides,
                       activation=None,
                       batch_normalization=False)
      x=keras.layers.add([x,y])
      x=Activation('relu')(x)
    num_filters *=2

    x=AveragePooling2D(pool_size=8)(x)
    y=Flatten()(x)
    outputs=Dense(num_classes,
                  activation='softmax',
                  kernel_initializer='he_normal')(y)

    model=Model(inputs=inputs,outputs=outputs)
    return model

In [ ]:
def resnet_v2(input_shape,depth, num_classes=10):
  if(depth-2)%9!=0:
    raise ValueError('depth should be 9n+2 (eg 56 orr 110 in [b])')
  num_filter_in=16
  num_res_blocks=int((depth-2)/9)
  x=resnet_layer(inputs=inputs,
                 num_filter=num_filter_in,
                 conv_first=True)
  for stage in range(3):
    for res_block in range(num_res_blocks):
      activation ='relu'
      batch_normalization =True
      strides=1
      if stage==0:
        num_filter_out=num_filter_in*4
        if res_block==0:
          activation=None
          batch_normalization==False
      else:
        num_filter_out=num_filter_in*2
        if res_block ==0:
          strides=2
      y=resnet_layer(inputs=x,
                     num_filter=num_filters_in,
                     kernel_size=1,
                     strides=strides,
                     activation=activation,
                     batch_normalization=batch_normalization,
                     conv_first=False)
      y=resnet_layer(inputs=y,
                     num_filters=num_filters_in,
                     conv_first=False)
      y=resnet_layer(inputs=y,
                     num_filters=num_filters_out,
                     kernel_size=1,
                     conv_first=False)
      if res_block==0:
        x=resnet_layer(inputs=x,
                       num_filter=num_filter_out,
                       kernel_size=1,
                       activation=None,
                       batch_normalization=False)
      x=keras.layers.add([x,y])
    num_filters_in=num_filters_out

    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    x=AveragePooling2D(pool_size=8)(x)
    y=Flatten()(x)
    outputs=Dense(num_classes,
                  activation='softmax',
                  kernel_initializer='he_normal')(y)

    mode=Model(inputs=inputs,outputs =outputs)
    return model

In [ ]:
if version ==2:
  model = resnet_v2(input_shape=input_shape,depth=depth)
else:
  model=resnet_v1(input_shape=input_shape,depth=depth)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

save_dir=os.path.join(os.getcwd(),'saved_models')
model_name='cifar10_10% s_model.{epoch:-3d}.h5'%model_type
if not os.path.isdir(save_dir):
  os.makedirs(save_dir)
filepath=os.path.join(save_dir,model_name)

checkpoint =ModelCheckpoint(filepath= filepath,
                            monitor='val_acc',
                            verbose=1,
                            save_best_only=True)
lr_scheduler=LearningRateScheduler(lr_schedule)
lr_reducer=ReduceLROnPlateau(factor=np.sqrt(0.1),
                             cooldown=0,
                             patience=5,
                             min_lr=0.5e-6)
callbacks=[checkpoint,lr_reducer, lr_scheduler]

if not data_augmentation:
  print('Not using data augmentation.')
  model.fit(x_train,y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test,y_test),
            shuffle=True,
            callbacks=callbacks)
else:
  print('Using real-time data augmentation.')
  datagen=ImageDataGenerator(
      featurewise_center=False,
      samplewise_center=False,
      featurewise_std_normalization=False,
      samplewise_std_normalization=False,
      zca_whitening=False,
      zca_epsilon=1e-06,
      rotation_range=0,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.,
      zoom_range=0.,
      channel_shift_range=0.,
      fill_mode='nearest',
      cval=0.,
      horizontal_flip=True,
      vertical_flip=False,
      rescale=None,
      preprocessing_function=None,
      data_format=None,
      validation_split=0.0
  )
  datagen.fit(x_train)
  model.fit_generator(datagen.flow(x_train,y_train,batch_size=batch_size),
                      validation_data=(x_test,y_test),
                      epochs=epochs, verbose=1,workers=4,
                      callbacks=callbacks)
  scores=model.evaluate(x_test,y_test,verbose=1)
  print('Test loss:', scores[0])
  print('Test accuracy:',scores[1])

Learning rate:  0.001
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 32, 32, 16)           448       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 32, 32, 16)           64        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 32, 32, 16)           0         ['ba

<ipython-input-7-0984372c8181>:61: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train,y_train,batch_size=batch_size),


1563/1563 [==============================] - ETA: 0s - loss: 1.5707 - accuracy: 0.4391

1563/1563 [==============================] - 383s 241ms/step - loss: 1.5707 - accuracy: 0.4391 - val_loss: 1.3198 - val_accuracy: 0.5289 - lr: 0.0010
Learning rate:  0.001
Epoch 2/200
1563/1563 [==============================] - ETA: 0s - loss: 1.2472 - accuracy: 0.5650

1563/1563 [==============================] - 353s 225ms/step - loss: 1.2472 - accuracy: 0.5650 - val_loss: 1.1756 - val_accuracy: 0.5883 - lr: 0.0010
Learning rate:  0.001
Epoch 3/200
1563/1563 [==============================] - ETA: 0s - loss: 1.1165 - accuracy: 0.6116

1563/1563 [==============================] - 348s 223ms/step - loss: 1.1165 - accuracy: 0.6116 - val_loss: 1.2476 - val_accuracy: 0.5862 - lr: 0.0010
Learning rate:  0.001
Epoch 4/200
1563/1563 [==============================] - ETA: 0s - loss: 1.0470 - accuracy: 0.6408

KeyboardInterrupt: 